# Import libraries

In [1]:
# !pip install unidecode
# !pip install emoji

In [2]:
import pandas as pd
import numpy as np
import datetime
from collections import Counter
import re
import emoji

# Load the training csv's and combine them into one dataframe

In [3]:
# load all 8 training csv's
df1 = pd.read_csv('reviews/train-1.csv')
df2 = pd.read_csv('reviews/train-2.csv')
df3 = pd.read_csv('reviews/train-3.csv')
df4 = pd.read_csv('reviews/train-4.csv')
df5 = pd.read_csv('reviews/train-5.csv')
df6 = pd.read_csv('reviews/train-6.csv')
df7 = pd.read_csv('reviews/train-7.csv')
df8 = pd.read_csv('reviews/train-8.csv')

# combine the dataframes into one big dataframe
df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8])

In [4]:
def remove_non_ascii_2(text):

    return re.sub(r'[^\x00-\x7F]',' ', text)

In [5]:
df['review_body'] = df['review_body'].apply(lambda x: x.replace('<br />', ' '))
# df['review_body'] = df['review_body'].apply(remove_non_ascii_2)

# Show the head of the data

In [6]:
df.head()

,Unnamed: 0,product_id,product_parent,product_title,vine,verified_purchase,review_headline,review_body,review_date,marketplace_id,product_category_id,label
0,9,B001N2MZT8,903886718,Green Zone [DVD],N,Y,green zone,I found at first it was a little difficult to ...,2010-11-15,1,3,False
1,11,B00GCBVE0Q,282740618,Le secret de Green Knowe,N,Y,NaN,J'ai aimé cette histoire. Les acteurs - et sur...,2014-11-23,2,3,False
2,19,1423165691,883799517,A Disney Sketchbook.,N,N,okay mais...,est-ce une coincidence que la plupart des prin...,2012-12-22,0,0,False
3,33,0061091480,623343977,Your Erroneous Zones,N,N,Arrogant,Wáyné Dyér is á pớpúlár áméricán pérsớnál grớw...,2009-07-21,0,0,True
4,34,B00HZ4CYOY,647510225,König der Mathematik Junior,N,Y,Tớllé Máthé Ápp...,.....unsere Kids mögen diese Art des Lernens. ...,2015-06-01,0,1,False


# Check if the unnamed column could be the review id

In [7]:
# print the number of rows and number of unique values in the unnamed column
# if they match, we can assume that it's the review_id
print(len(df['Unnamed: 0'].unique()), len(df))

9614 9614


# Most frequent values analysis
This could find default or placeholder values

In [8]:
# iterate over the columns and print the most frequent values
for label, content in df.items():
    print(label)
    print(content.mode())
    print()

Unnamed: 0
0           1
1           2
2           3
3           4
4           5
        ...  
9609    11993
9610    11994
9611    11995
9612    11998
9613    11999
Length: 9614, dtype: int64

product_id
0    B00FAPF5U0
dtype: object

product_parent
0    600633062
dtype: int64

product_title
0    Gone Girl
dtype: object

vine
0    N
dtype: object

verified_purchase
0    Y
dtype: object

review_headline
0    Five Stars
dtype: object

review_body
0    Great
dtype: object

review_date
0    2015-01-02
dtype: object

marketplace_id
0    0
dtype: int64

product_category_id
0    3
dtype: int64

label
0    False
dtype: bool



# Unique values analysis
This could find broken values in columns with just a couple of unique values.

In [9]:
# iterate over the columns and get the unique values
for label, content in df.items():
    print(label)
    print(content.unique())
    print()

Unnamed: 0
[    9    11    19 ... 11951 11953 11998]

product_id
['B001N2MZT8' 'B00GCBVE0Q' '1423165691' ... 'B000026FJ6' 'B00K6S2HS4'
 'B00B1COIRC']

product_parent
[903886718 282740618 883799517 ... 475533906  96406179 716679008]

product_title
['Green Zone [DVD]' 'Le secret de Green Knowe' 'A Disney Sketchbook.' ...
 'The Songs of Distant Earth' 'Tolles Album' 'Populaire']

vine
['N' 'Y']

verified_purchase
['Y' 'N']

review_headline
['green zone' nan 'okay mais...' ... 'Native (Gold Edition)'
 'Coup de coeur pour ce film adorable et original' 'Dans la lignée du 1er']

review_body
["I found at first it was a little difficult to associate Matt Damon as anything else but Jason Bourne.But as the film settled in I forgot Jason.The story wasn't overely bright but was conclusive as to what it was all about.That is the Americans covering a story which has already been told.i.e The Big Cover Up,of the Iraq affair.Overall I thought it was a good film , forget the politics watch the film.Good

# Check the rows with NaN values
The unique values analysis showed that the review_headline could be NaN, so we check the whole dataframe on NaN values. It seems that 715 rows don't have a review headline.

In [10]:
# show the rows with null in the 
df[df['review_headline'].isnull()]

,Unnamed: 0,product_id,product_parent,product_title,vine,verified_purchase,review_headline,review_body,review_date,marketplace_id,product_category_id,label
1,11,B00GCBVE0Q,282740618,Le secret de Green Knowe,N,Y,NaN,J'ai aimé cette histoire. Les acteurs - et sur...,2014-11-23,2,3,False
7,49,B00005JYAX,581643936,Memento,N,N,NaN,"Le film est excellent niveau suspense,... Je l...",2002-09-28,0,3,True
35,383,B000024611,769213394,Phil Collins...Hits,N,Y,NaN,Un peu trop mou ce ne sont pas les versions le...,2014-08-29,2,3,False
62,642,B000050FII,676028219,Fight Club [Édition Collector],N,N,NaN,"Un film comme l'on en voit désormais très peu,...",2000-12-02,2,3,False
78,912,B009CAMJWQ,779021042,Jason Bourne : L'Héritage [Edition Limitée - B...,N,N,NaN,Un peu déçu par cet épisode qui n'est pas à la...,2013-08-14,2,3,True
...,...,...,...,...,...,...,...,...,...,...,...,...
1084,11277,B005FGMBY2,558593095,Les Frères Grimm [Blu-ray],N,Y,NaN,Selon les gouts me direz-vous... C'est vrai et...,2014-12-11,2,3,False
1122,11631,B00AFBJIVQ,178375143,Les cinq légendes,N,Y,NaN,J'étais plutot retissante d'aller voir ce film...,2013-05-11,2,3,True
1129,11706,B00FAC2GKA,383229535,Brain Salad Surgery,N,Y,NaN,Je suis extrêmement déçu.. Cette édition conti...,2014-07-25,2,6,True
1131,11791,B0058HDHAK,766029498,One (Remasterisé),N,Y,NaN,C'est une compilation incontournable et qui pe...,2014-02-08,2,6,False


# Check which columns have NaN values
Other columns could also contain NaN values, so this is checked below. This way we know which columns we might have to fix.

In [11]:
# iterate over the columns and check if any row has NaN for this column
for label, content in df.items():
    print(label)
    print(content.isnull().any())
    print()

Unnamed: 0
False

product_id
False

product_parent
False

product_title
True

vine
False

verified_purchase
False

review_headline
True

review_body
False

review_date
True

marketplace_id
False

product_category_id
False

label
False



# Group the data on the product_id column
We expect the products to have the same product title, product parent and product category_id

In [12]:
# group the data on the product id 
df_gb = df.groupby('product_id')

## Get the products with multiple product titles

In [13]:
# get the counts and number of unique values per product_id group
aggs = df_gb.agg(['nunique']).reset_index(drop=False)

# print all rows with more than one unique product title
multiple_titles = aggs[aggs['product_title','nunique'] > 1]['product_id']

# most common titles of product ids with multiple titles 
titles= [[x, df_gb.get_group(x)['product_title'].mode()] for x in multiple_titles]
#titles

### Interesting data:
I think that the spacing in front of the titles is a weird print thing.

```
 0                                              Awesome
 1    Harry Potter, volume 1: Harry Potter and the P...
 dtype: object,
 
 0    American IV : The Man Comes Around
 1    American IV:  The Man Comes Around
 dtype: object,
 
 0    Sticks  + Stones
 1     Sticks + Stones
 dtype: object,
 
 0           Obscured By Clouds (remastered)
 1    Obscured By Clouds [Discovery Edition]
 dtype: object,
 
 0    Doctor Who - Complete Series 1-4 Box Set [23 D...
 1            Doctor Who: Series 1 - 4 Collection [DVD]
 dtype: object,
 
 0     Pink - Funhouse Tour/Live in Australia [Blu-ray]
 1    Pink - Funhouse Tour: Live In Australia [Blu-ray]
 dtype: object,
 
 0    Good Evening New York City [2 CD + 1 DVD]
 1      Good evening New York City (2CD + 1DVD)
 dtype: object,
 
 0                     Amazing
 1    Spánnénd bis zúm Schlúss
 2            Zwei von Drei...
 dtype: object,
 
 0     FarmVille 2: Country Escape
 1    FarmVille 2: Raus auf's Land
 dtype: object,
```


In [14]:
# checking the rows of a product
df[df['product_id'] == 'B0024RID60']

,Unnamed: 0,product_id,product_parent,product_title,vine,verified_purchase,review_headline,review_body,review_date,marketplace_id,product_category_id,label
723,7116,B0024RID60,849107619,Some Girls,N,N,Die letzte große Platte der Stones,Ich kann mich noch sehr gut an die damaligen P...,2011-10-23,0,6,True
458,4659,B0024RID60,849107619,Remarquable !!!,N,N,Some Girls,"Attention, chef d'oeuvre : Some Girls, sorti e...",2011-02-08,2,6,True


# Get the rows with the switched product_title and review_headline

In [15]:
# most common titles of product ids with multiple titles 
unique_titles = [[x, df_gb.get_group(x)['product_title'].unique()] for x in multiple_titles]
#unique_titles

In [16]:
def fix_title_headline(df, row):
    
    # get the possible titles of the product id of this row
    rows = df[df['product_id'] == row['product_id']]
    
    # the unique titles of this product
    unique_titles = rows['product_title'].unique()
    
    # get the review headline
    review_headline = row['review_headline']
    
    # check if the review_headline is in the unique titles
    if review_headline in unique_titles:
        
        # replace the review headline of the row with the title
        row['review_headline'] = row['product_title']
        
        # replace the title of the row with the review headline
        row['product_title'] = review_headline
        
    # return the modified row
    return row

In [17]:
df.apply(lambda x: fix_title_headline(df, x), axis=1)

,Unnamed: 0,product_id,product_parent,product_title,vine,verified_purchase,review_headline,review_body,review_date,marketplace_id,product_category_id,label
0,9,B001N2MZT8,903886718,Green Zone [DVD],N,Y,green zone,I found at first it was a little difficult to ...,2010-11-15,1,3,False
1,11,B00GCBVE0Q,282740618,Le secret de Green Knowe,N,Y,NaN,J'ai aimé cette histoire. Les acteurs - et sur...,2014-11-23,2,3,False
2,19,1423165691,883799517,A Disney Sketchbook.,N,N,okay mais...,est-ce une coincidence que la plupart des prin...,2012-12-22,0,0,False
3,33,0061091480,623343977,Your Erroneous Zones,N,N,Arrogant,Wáyné Dyér is á pớpúlár áméricán pérsớnál grớw...,2009-07-21,0,0,True
4,34,B00HZ4CYOY,647510225,König der Mathematik Junior,N,Y,Tớllé Máthé Ápp...,.....unsere Kids mögen diese Art des Lernens. ...,2015-06-01,0,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...
1147,11940,B0011V9XT6,475533906,Beyoncé: The Beyonce Experience - Live [Blu-ra...,N,N,Blu-ray definitely all regions,I don't know why Amazon and one reviewer here ...,2009-12-29,1,3,False
1148,11944,B000026FJ6,96406179,The Songs of Distant Earth,N,Y,Wohlklingende Entspannung,"Ich mag diese Musik sehr, es trifft genau mein...",2013-12-18,0,6,True
1149,11951,B00K6S2HS4,409414789,Tolles Album,N,Y,Native (Gold Edition),"Tolle Band, tolles Album, tolle Lieder, einfac...",2015-02-23,0,6,False
1150,11953,B00B1COIRC,716679008,Populaire,N,N,Coup de coeur pour ce film adorable et original,R&eacute;jouissante com&eacute;die grâce &agra...,2015-04-10,0,3,True


## Get the products with multiple product parents

In [18]:
# print all rows with more than one unique product parent
multiple_parents = aggs[aggs['product_parent','nunique'] > 1]['product_id']

# most common parents of product ids with multiple parents 
parents = [df_gb.get_group(x)['product_parent'].mode() for x in multiple_parents]
parents

[]

## Get the products with multiple product categories
0 means it's a book, not that it's missing.

In [19]:
# print all rows with more than one unique product category
multiple_categories = aggs[aggs['product_category_id','nunique'] > 1]['product_id']

# print unique category ids of product ids with multiple categories 
categories = [df_gb.get_group(x)['product_category_id'].unique() for x in multiple_categories]
#categories

# Check the marketplace languages
I saw some review bodies with [escaped characters](https://fantasai.tripod.com/FHQR/Characode/EscCodes.htm), all normal vowels with an accent aigu, random &-signs and html line breaks. The languages of marketplace 1, 2 and 3 are correct. Marketplace 0 contains all 3 languages. We might want to try to predict the language so that we could assign them to a marketplace.

In [20]:
# show the review headline and body of a certain markteplace id 
# 0 = null/no particular marketplace 
# 1 = english 
# 2 = french 
# 3 = german
df[df['marketplace_id'] == 3][['review_headline','review_body']]

,review_headline,review_body
33,Ein Klassiker,Ein Muss für jede Plattensammlung. Geht richti...
117,"alles drin, was man braucht","Für den unschlagbaren Preis von 12,97 Euro ein..."
153,Slash,Dá méiné Vớrrédnér dié Détáils béléúchtét hábé...
304,Star Trek Into Darkness (Blu-ray 3D + DVD + Di...,Spannend wie der erste Star Trek 11. Action ge...
384,Minútés tớ Midnight-Dié Kúnst: Linkin Párks sc...,Da in den vergangenen Monaten viele Fake-Alben...
...,...,...
938,Placebo Loud like Love,Ich bin total begeistert von dieser CD. Man ka...
969,Gut!,Guter Thriller in dem man Tom Cruise sogar den...
1027,außergewöhnlich in Stimmer und Lied,Loreen ist einfach Spitzenklasse. Ich höre sie...
1066,Éinfách núr Súpér.,"Bis auf ein oder zwei Stücke, die arg schnulzi..."


# Check if there are invalid date values

In [21]:
# funtion that tries to make a datetime from the string
# true means valid, false means invalid, None means NaN
def vali_date(date_text):
    try:
        if not isinstance(date_text, str) and np.isnan(date_text): return None
        datetime.datetime.strptime(date_text, '%Y-%m-%d')
        return True
    except ValueError:
        return False

In [22]:
# get the valid dates (True, False or None) and count them
vali_dates = [vali_date(x) for x in df['review_date']]
Counter(vali_dates)

Counter({True: 9613, None: 1})

# Check which accented characters are in the review body

In [23]:
def special_chars(review_body):
    
    review_body = str(review_body)
    
    # the number of accents
    asciis = ''
    
    # iterate over the characters and check if acute is in their unicode name
    for char in review_body:
        
        if len(char.encode()) > 1: asciis += (char)
            
    # return the ratio of non ascii characters
    return asciis

In [24]:
a = df['review_body'].apply(special_chars)

In [25]:
Counter(''.join(a)).most_common(10)

[('é', 81445),
 ('á', 25552),
 ('ú', 16993),
 ('ớ', 13862),
 ('ü', 7376),
 ('ä', 6088),
 ('ö', 3828),
 ('à', 3592),
 ('è', 3367),
 ('Á', 2192)]

## Check per marketplace

In [26]:
# group by marketplace
df_mp = df.groupby('marketplace_id')['review_body']

In [27]:
# iterate over the groups
for group_id, group in df_mp:
    
    # get the special characters
    result = group.apply(special_chars)
    
    # print the most common special characters per marketplace group 
    print(group_id, Counter(''.join(result)).most_common(10))

0 [('é', 65838), ('á', 24264), ('ú', 16169), ('ớ', 13153), ('ü', 7008), ('ä', 5793), ('ö', 3598), ('Á', 2076), ('ß', 1861), ('É', 1516)]
1 [('’', 53), ('£', 30), ('”', 24), ('“', 22), ('‘', 14), ('é', 10), ('´', 5), ('😁', 5), ('😜', 3), ('😝', 3)]
2 [('é', 12226), ('à', 2633), ('è', 2441), ('ê', 1430), ('ç', 720), ('’', 580), ('ô', 427), ('î', 248), ('â', 216), ('û', 176)]
3 [('é', 3371), ('á', 1288), ('ú', 822), ('ớ', 709), ('ü', 354), ('ä', 289), ('ö', 225), ('Á', 116), ('ß', 90), ('É', 62)]


# Get a good threshold for the accent spam in the review body
It seems like 0.11 is a good threshold

In [28]:
def non_ascii_ratio(review_body):
    
    review_body = str(review_body)
    
    # the total number of characters
    total = len(review_body)
    
    # the number of accents
    asciis = 0
    
    # iterate over the characters and check if acute is in their unicode name
    for char in review_body:
        
        if len(char.encode()) > 1:
            asciis += 1
            
    # return the ratio of non ascii characters
    return asciis/total

In [29]:
# copy the dataframe, add the non ascii ratio and show the rows from a certain threshold (it shows that)
df_copy = df.copy()
df_copy['non_ascii_ratio'] = df_copy['review_body'].apply(non_ascii_ratio)
df_copy = df_copy.sort_values('non_ascii_ratio')
df_copy[df_copy['non_ascii_ratio'] > 0.07].head()

,Unnamed: 0,product_id,product_parent,product_title,vine,verified_purchase,review_headline,review_body,review_date,marketplace_id,product_category_id,label,non_ascii_ratio
897,8809,B00AFBJJJW,178375143,Les Cinq légendes [Combo Blu-ray 3D + Blu-ray ...,N,Y,NaN,Un vrai chef d’œuvre visuel. L'histoire est tr...,2013-04-11,0,3,False,0.075862
1187,11618,B009IA9OLE,278434220,Devil's Attorney,N,Y,NaN,"Très bon, il fait passer le temps mais très tr...",2014-07-05,2,1,False,0.088235
348,3706,B00QW8TYWO,828652896,Crossy Road,N,Y,AMAZING,When I first played this game it was AMAZING a...,2015-02-05,1,1,False,0.109677
734,7226,B00GU40HWQ,430605778,Sớớớớớớ Cớớớớớớlllllll !!!!!!!!!!!,N,Y,Criminal Minds - Die komplette achte Staffel [...,Ist dié Géilsté Sérié dér Wélt !!!!! !!!!!! !!...,2014-02-16,0,3,True,0.119048
712,7121,B007SVESDO,992326428,Marvel's The Avengers,N,N,hmhmmmmm....,"... ich vérstéhé nicht, wárúm diésér film núr ...",2012-06-02,0,3,True,0.146939


In [30]:
# check the review body of the row around the possible threshold that doesn't have a clear accent spam
df[df['Unnamed: 0'] == 3706]['review_body'].values[0]

'When I first played this game it was AMAZING and now no have nearly collected all the characters my favourite is the flea and the universe😜😜😜😝😝😝😍😍😍😎😘😘😘😻😻😸😸'

In [31]:
import unidecode
accented = df[df['Unnamed: 0'] == 7226]['review_body'].values[0]
print(accented)
unaccented = unidecode.unidecode(accented)
print(unaccented)

Ist dié Géilsté Sérié dér Wélt !!!!! !!!!!! !!!!! &#60;3 &#60;3  Liébé Grüßé  Jénnifér Kérn &#60;3 &#60;3 &#60;3 &#60;3 &#60;3
Ist die Geilste Serie der Welt !!!!! !!!!!! !!!!! &#60;3 &#60;3  Liebe Grusse  Jennifer Kern &#60;3 &#60;3 &#60;3 &#60;3 &#60;3


## Only focus on the acute accent which seems to be the only accent used for spam

In [32]:
def acute_ratio(review_body):
    
    review_body = str(review_body)
    
    # the total number of characters
    total = len(review_body)
    
    # the number of accents
    acutes = 0
    
    # the list of char
    chars = ['á', 'Á', 'é', 'É', 'í', 'Í', 'ớ', 'ó', 'Ó', 'ú', 'Ú']
    
    # iterate over the characters and check if acute is in their unicode name
    for char in review_body:
        
        if char in chars:
            acutes += 1
            
    # return the ratio of non ascii characters
    return acutes/total

In [33]:
# copy the dataframe, add the non ascii ratio and show the rows from a certain threshold (it shows that)
df_copy = df.copy()
df_copy['acute_ratio'] = df_copy['review_body'].apply(acute_ratio)
df_copy = df_copy.sort_values('acute_ratio')
df_copy[df_copy['acute_ratio'] > 0.045].head(10)

,Unnamed: 0,product_id,product_parent,product_title,vine,verified_purchase,review_headline,review_body,review_date,marketplace_id,product_category_id,label,acute_ratio
862,8437,B008NBEGV2,449129606,City Hall,N,N,NaN,Casting alléchant ... mais au final film qui p...,2013-03-10,2,1,False,0.045249
660,6780,B00H3GZMIE,365698290,S.SMITH-IN THE LONELY HOUR CD,N,Y,Tout simplement génial !,Tout simplement génial ! A écouter et réécoute...,2015-04-30,0,3,False,0.047244
96,880,2070512541,521556475,Matilda,N,N,NaN,Article conforme à la commande et délai respec...,2009-08-09,2,0,True,0.048544
655,6758,B004ITWEHQ,38673835,The unforgiving,N,Y,Très bonne surprise!,C'est une découverte récente et réussie. Pas t...,2014-12-26,2,6,False,0.048611
327,3332,B000274SH2,198816259,Tout peut arriver,N,Y,produit abimé,"le produit a été dégradé pendant le transport,...",2013-12-16,0,3,False,0.049430
155,1396,B008P1EWIM,67489912,Météo Live - des Prévisions Détaillées et des ...,N,Y,NaN,C'était excellent... A plu énormément. Je le r...,2013-03-01,2,1,True,0.051613
734,7226,B00GU40HWQ,430605778,Sớớớớớớ Cớớớớớớlllllll !!!!!!!!!!!,N,Y,Criminal Minds - Die komplette achte Staffel [...,Ist dié Géilsté Sérié dér Wélt !!!!! !!!!!! !!...,2014-02-16,0,3,True,0.103175
712,7121,B007SVESDO,992326428,Marvel's The Avengers,N,N,hmhmmmmm....,"... ich vérstéhé nicht, wárúm diésér film núr ...",2012-06-02,0,3,True,0.142857
69,592,B00005AAFS,247505567,Mutter,N,N,Laut und lässig,"Éigéntlich höré ich Jázz, ábér diésé Cd ist Pf...",2001-11-24,0,6,False,0.142857
521,5234,B0035KGWR2,645291585,A Curious Thing,N,Y,cd,"séhr schöné Músik , émpféhléndswért únd éin sc...",2011-01-11,0,6,False,0.146341


In [40]:
def replace_acute_accents(text, accent_map):
    for [accent, char] in accent_map:
        text = re.sub(accent, char, text)
    return text

def remove_accent_spam(text, threshold=0.07):
    
    # the total number of characters
    total = len(text)
    
    # the number of accents
    acutes = 0
    
    # the map of acute char with their non accented variant
    acute_map = np.array([['á', 'a'], ['Á', 'A'], ['é', 'e'], ['É', 'E'], 
        ['í', 'i'], ['Í', 'I'], ['ớ', 'o'], ['ó', 'o'], 
        ['Ó', 'O'], ['ú', 'u'], ['Ú', 'U']])
    
    # iterate over the characters and check if acute is in their unicode name
    for char in text:
        if char in acute_map[:, 0]:
            acutes += 1
            
    # the ratio of non ascii characters
    ratio = acutes/total
    
    # just return the original review body if the ratio does not exceed the threshold
    if ratio <= threshold: return text
    
    # return the decoded text
    return replace_acute_accents(text, acute_map)

In [41]:
df_copy['review_body'] = df_copy['review_body'].apply(remove_accent_spam)

In [44]:
df_copy['acute_ratio'] = df_copy['review_body'].apply(acute_ratio)
df_copy = df_copy.sort_values('acute_ratio')
df_copy.tail()

,Unnamed: 0,product_id,product_parent,product_title,vine,verified_purchase,review_headline,review_body,review_date,marketplace_id,product_category_id,label,non_ascii_ratio,acute_ratio
660,6780,B00H3GZMIE,365698290,S.SMITH-IN THE LONELY HOUR CD,N,Y,Tout simplement génial !,Tout simplement génial ! A écouter et réécoute...,2015-04-30,0,3,False,0.041667,0.047244
96,880,2070512541,521556475,Matilda,N,N,NaN,Article conforme à la commande et délai respec...,2009-08-09,2,0,True,0.000000,0.048544
655,6758,B004ITWEHQ,38673835,The unforgiving,N,Y,Très bonne surprise!,C'est une découverte récente et réussie. Pas t...,2014-12-26,2,6,False,0.050000,0.048611
327,3332,B000274SH2,198816259,Tout peut arriver,N,Y,produit abimé,"le produit a été dégradé pendant le transport,...",2013-12-16,0,3,False,0.076923,0.049430
155,1396,B008P1EWIM,67489912,Météo Live - des Prévisions Détaillées et des ...,N,Y,NaN,C'était excellent... A plu énormément. Je le r...,2013-03-01,2,1,True,0.000000,0.051613


## Try to get the texts that do not contain unaccented characters
Seems to be the best/most reliable solution

In [61]:
def normal_vowels(review_body):
    
    # make sure it is a string
    review_body = str(review_body)
    
    # the map of acute char with their non accented variant
    acute_map = np.array([['á', 'a'], ['Á', 'A'], ['é', 'e'], ['É', 'E'], 
        ['ớ', 'o'], ['ó', 'o'], ['Ó', 'O'], ['ú', 'u'], ['Ú', 'U']])
    
    # iterate over the characters and check if acute is in their unicode name
    for char in review_body:
        
        if char in acute_map[:, 1]:
            return 0
            
    # return the ratio of non ascii characters
    return 1

In [64]:
# copy the dataframe, add the non ascii ratio and show the rows from a certain threshold (it shows that)
df_copy = df.copy()
df_copy['normal_vowels'] = df_copy['review_body'].apply(normal_vowels)
df_copy = df_copy.sort_values('normal_vowels')
df_copy.tail(10)

,Unnamed: 0,product_id,product_parent,product_title,vine,verified_purchase,review_headline,review_body,review_date,marketplace_id,product_category_id,label,n_emojis,normal_vowels
414,3915,3526434670,999720718,The Grapes of Wrath,N,Y,hábé dén ártikél nié békớmmén.,"múss gléich náchfớrschén, wás mit dém Ártikél ...",2014-02-13,0,0,False,0,1
589,6061,B00ABA1OOK,806256911,TinkerBell: Das Geheimnis der Feenflügel,N,Y,Tinkerbell...,Éin tớllér Fiilm für dié gánzé Fámilié. Ich gé...,2013-03-21,0,11,False,0,1
513,5305,B00VS8WIY4,401487716,Cisco Bandits: A Gwynn Reznick Mystery (Gwynn ...,N,N,Thrilling story,Ciscớ Bándits: Á Gwynn Réznick Nớvél ... Ingé-...,2015-04-20,0,4,False,0,1
496,5016,B00CT2P6VA,622332427,Quell Memento,N,Y,Ein sch&ouml;nes Game,Dás Spiél ist génáú sớ sch&ớúml;n wié dié ándé...,2013-07-18,0,1,True,0,1
298,2810,B0088UPIMY,495882719,Findet Nemo [Blu-ray],N,Y,bd findet nemo,zúm film múss mán nicht viél ságén. bild únd t...,2013-09-02,0,3,False,0,1
439,4476,B005CNZN7A,824349893,Prima Qualität,N,Y,Harry Potter und die Heiligtümer des Todes - T...,Állés primá géláúfén súpér Qúálität.gérné wiéd...,2012-12-09,0,3,False,0,1
519,5334,B009I2UH2W,518209625,Griffin GC36547 KFZ-Ladekabel PowerJolt SE mit...,N,Y,Ersguterjunge19,Qúálität ist séhr gút! Léidér kéiné Infớ in dé...,2015-01-01,0,17,False,0,1
54,510,B00005UE6T,194578547,Requiem for a Dream,N,N,Psychisch mitreißend,Mit Ábstánd dér bésté Drớgénfilm dén ich jéh g...,2004-01-05,0,3,True,0,1
852,8432,B00AGKH7K0,664578732,'Til Your River Runs Dry,N,Y,Ein Altmeister!,"Mit séinér Músik bin ich áúfgéwáchsén, únd héú...",2014-01-27,0,6,False,0,1
900,8777,B00N57VBRG,621648805,Leo's Fortune,N,Y,Super,Dás Spiél ist mit viél Liébé éntwickélt. Dié G...,2014-11-21,0,1,False,0,1


# Get a good threshold for the accent spam in the review headline
It seems like 0.11 is a good threshold for any language but French, we might want to keep that in mind

In [35]:
# copy the dataframe, add the non ascii ratio and show the rows from a certain threshold (it shows that)
df_copy = df.copy()
df_copy['non_ascii_ratio'] = df_copy['review_headline'].apply(non_ascii_ratio)
df_copy = df_copy.sort_values('non_ascii_ratio')
df_copy[(df_copy['non_ascii_ratio'] > 0.11) & (df_copy['marketplace_id'] != 2)].head()

,Unnamed: 0,product_id,product_parent,product_title,vine,verified_purchase,review_headline,review_body,review_date,marketplace_id,product_category_id,label,non_ascii_ratio
949,9317,B006OOHYIC,618772576,Quell Reflect,N,Y,Kniffélig,Éin knifféligés únd éntspánnéndés Spiél mit Sú...,2013-07-14,0,1,False,0.111111
832,8669,B005C3HBHU,167297708,Jeeves and Wooster - The Complete Collection [...,N,Y,Göttlich~,Álléiné dié érsté Épisớdé wár jédén Cént wért....,2014-04-26,0,3,True,0.111111
1053,10416,B000071IY2,982206081,Star Trek 10 - Nemesis,N,Y,DVD lässt zu wünschen übrig,"Die einen wollen Action, die anderen Philosoph...",2003-07-28,0,11,True,0.111111
476,4906,B00G6SDX36,140219026,Spigen Ultra Fit Coque pour iPhone 5/5S Transp...,N,Y,Très bonne qualité,"La Coque est de très bonne qualité, elle est f...",2015-08-11,0,17,False,0.111111
599,5889,B000025W7O,824346667,Toto IV,N,N,Ihr größter Erfolg,Im April 1982 erschien das vierte Album der U....,2015-05-29,0,6,True,0.111111


In [49]:
# copy the dataframe, add the non ascii ratio and show the rows from a certain threshold (it shows that)
df_copy = df.copy()
df_copy['acute_ratio'] = df_copy['review_headline'].apply(acute_ratio)
df_copy = df_copy.sort_values('acute_ratio')
df_copy[df_copy['acute_ratio'] > 0.12].head(10)

,Unnamed: 0,product_id,product_parent,product_title,vine,verified_purchase,review_headline,review_body,review_date,marketplace_id,product_category_id,label,n_emojis,acute_ratio
1171,11712,B000001GXF,683310455,"Schubert : Quintette ""La Truite"" - Quatuor ""La...",N,N,Décapant,Est-il besoin de dire qu'entre Gilels et les A...,2010-11-24,0,6,True,0,0.125
157,1707,B00QM9FRT2,661195539,Guardians of the Galaxy [dt./OV],N,Y,béstér Film 2014,Super Marvel Film. Mega Effekte aber vor allem...,2015-01-05,0,9,False,0,0.125
483,4873,B0019IB26I,208274744,Where the Light Is:Live l.a. [Import anglais],N,Y,décevant,"acheté suite aux commentaires élogieux, et bie...",2014-10-12,2,6,False,0,0.125
653,6750,B00005QDW1,746338180,Rock Steady,N,N,Ziémich gút.....,...finde ich.<BR>Allerdings muss ich dazu sage...,2002-06-14,0,6,True,0,0.125
981,9639,B00004VYNZ,848009018,Wild Wild West,N,Y,Agréable,J'adore Will Smith. C'est bon acteur. Je l'ai ...,2012-10-02,2,3,False,0,0.125
1218,11700,B008MZGJXM,73561444,The Truth About Love (Deluxe Edition),N,Y,P!nk in Béstfớrm,béim érstén Hörén ... dér érsté Éindrúck: éin ...,2012-09-25,0,6,True,0,0.125
198,1918,B00KA7JIII,897384773,Monument Valley,N,Y,Génial !,Ce jeu est tout simplement génial ! Plein de p...,2015-02-09,2,5,False,0,0.125
887,8670,B000A5DV42,824202515,Road to Rouen,N,N,Réussite,"Supergrass l'avait annoncé, \\""l'album acousti...",2005-09-22,2,6,True,0,0.125
1080,10844,B001HUGYB6,869135660,Das Haus am See,N,Y,Tớp Film,"Dieser Film kann ich nur empfehlen, die Qualit...",2014-08-11,0,3,True,0,0.125
361,3556,B00DSKW9EO,315507905,Very Bad Trip 3,N,Y,Médiocre,Le moins bon des trois. Tout le monde s'essouf...,2014-06-01,2,3,False,0,0.125


# Emoji analysis

In [36]:
def extract_emojis(text):
    return ''.join([char for char in text if char in emoji.UNICODE_EMOJI['en']])

In [37]:
# create a new column with the emoji count
df['n_emojis'] = df['review_body'].apply(lambda x: len(extract_emojis(x)))

In [38]:
# print the number of rows that have at least 1 emoji
df[df['n_emojis'] > 0]['n_emojis'].count()

17